# Rasterizing a voronoi diagram

In [789]:
from PIL import Image, ImageDraw
from scipy.spatial import Voronoi, voronoi_plot_2d
import matplotlib.pyplot as plt
import numpy as np

In [790]:
def center_ellipse(x,y,r,c):

    draw.ellipse([x - r, y - r, x + r, y + r],fill=c)

def center_rectangle(x,y,l,w,c):
    l = l/2
    w = w/2
    draw.rectangle([x - w, y - l, x + w, y + l],fill=c)

Drawing a grid of pixels

In [791]:
res = 100
image_resolution = 1080
offset = 0.5
scale_factor = image_resolution / res

img = Image.new('RGB', (image_resolution, image_resolution))
draw = ImageDraw.Draw(img)

for x in range(res):
    for y in range(res):
        center_rectangle((x + 0.5) * scale_factor, (y + 0.5) * scale_factor, scale_factor - 2, scale_factor - 2, 'blue')
# draw.rectangle([0,0,image_resolution,image_resolution],fill="white")
img.save('pixels.png')

Rasterizing a single polygon

In [792]:
triangle = np.array([[15,10],[75,20],[60,60]])

# for vert in triangle:
#     x = vert[0] * scale_factor
#     y = vert[1] * scale_factor
#     center_ellipse(x,y,10,'white')

# img.save('pixels.png')

Edge function

In [793]:
def Edge_Function(_v0, _v1, _p):
    #Assumes clockwise order
    result = (_p[0] - _v0[0]) * (_v1[1] - _v0[1]) - (_p[1] - _v0[1]) * (_v1[0] - _v0[0])
    
    if result <= 0:
        return True
    else:
        return False

Single edge case

In [794]:
img = Image.open('pixels.png')
draw = ImageDraw.Draw(img)

for x in range(res):
    for y in range(res):
        state = Edge_Function(triangle[0],triangle[1],[x,y])
        if state:
            center_rectangle((x + 0.5) * scale_factor, (y + 0.5) * scale_factor, scale_factor - 2, scale_factor - 2, 'blue')
        else:
            center_rectangle((x + 0.5) * scale_factor, (y + 0.5) * scale_factor, scale_factor - 2, scale_factor - 2, 'red')
img.save('pixels.png')

Rasterizing a triangle

In [795]:
img = Image.open('pixels.png')
draw = ImageDraw.Draw(img)
op_count = 0
for x in range(res):
    for y in range(res):
        edge0 = Edge_Function(triangle[0],triangle[1],[x,y])
        edge1 = Edge_Function(triangle[1],triangle[2],[x,y])
        edge2 = Edge_Function(triangle[2],triangle[0],[x,y])
        
        
        if edge0 and edge1 and edge2:
            center_rectangle((x + 0.5) * scale_factor, (y + 0.5) * scale_factor, scale_factor - 2, scale_factor - 2, 'red')
        else:
            center_rectangle((x + 0.5) * scale_factor, (y + 0.5) * scale_factor, scale_factor - 2, scale_factor - 2, 'blue')
        op_count += 1
img.save('pixels.png')
print(f"total operations: {op_count}")

total operations: 10000


Optimizing by reducing pixel search to bounding box

In [796]:
def Bounding_Box(triangle):
    x_vals = []
    y_vals = []
    for vert in triangle:
        x_vals.append(vert[0])
        y_vals.append(vert[1])

    x_min = np.min(x_vals)
    x_max = np.max(x_vals) + 1 #account for
    y_min = np.min(y_vals)
    y_max = np.max(y_vals) + 1 #account for

    return ([x_min, x_max],[y_min,y_max])

img = Image.open('pixels.png')
draw = ImageDraw.Draw(img)

bbox = Bounding_Box(triangle)
op_count = 0
print(bbox)
for x in range(*bbox[0]):
    for y in range(*bbox[1]):
            
            edge0 = Edge_Function(triangle[0],triangle[1],[x,y])
            edge1 = Edge_Function(triangle[1],triangle[2],[x,y])
            edge2 = Edge_Function(triangle[2],triangle[0],[x,y])
            
            
            if edge0 and edge1 and edge2:
                center_rectangle((x + 0.5) * scale_factor, (y + 0.5) * scale_factor, scale_factor - 2, scale_factor - 2, 'red')
            else:
                center_rectangle((x + 0.5) * scale_factor, (y + 0.5) * scale_factor, scale_factor - 2, scale_factor - 2, 'blue')
                center_ellipse((x + 0.5) * scale_factor, (y + 0.5) * scale_factor, 2, 'white')
            op_count += 1
img.save('pixels.png')
print(f"total operations: {op_count}")


([15, 76], [10, 61])
total operations: 3111


Rasterizing a Polygon

Must be convex & ordered

In [797]:
polygon = np.array([[15,10],[75,20],[80,40],[70,80],[50,90],[10,50]])

img = Image.open('pixels.png')
draw = ImageDraw.Draw(img)


print(bbox)

def Rasterize_Polygon(polygon, color, background):

    
    bbox = Bounding_Box(polygon)

    for x in range(*bbox[0]):
        for y in range(*bbox[1]):
                state = True
                vertex_count = len(polygon)

                for i in range(vertex_count):
                    v0 = i
                    v1 = i + 1
                    if v1 == vertex_count:
                        v1 = 0
                    edge_state = Edge_Function(polygon[v0],polygon[v1],[x,y])
                    state = state and edge_state
                
                # if edge0 and edge1 and edge2 and edge3 and edge4 and edge5:
                if state:
                    center_rectangle((x + 0.5) * scale_factor, (y + 0.5) * scale_factor, scale_factor - 2, scale_factor - 2, color)
                else:
                    center_rectangle((x + 0.5) * scale_factor, (y + 0.5) * scale_factor, scale_factor - 2, scale_factor - 2, background)
                    center_ellipse((x + 0.5) * scale_factor, (y + 0.5) * scale_factor, 2, 'white')



    img.save('pixels.png')

for vert in polygon:
    x = vert[0] * scale_factor
    y = vert[1] * scale_factor
    center_ellipse(x,y,10,'white')

Rasterize_Polygon(polygon,'red','blue')

([15, 76], [10, 61])


Sorting a polygon:

In [798]:
unsorted_polygon = np.array([[75,20],[80,40],[15,10],[70,80],[50,90],[10,50]])

img = Image.open('pixels.png')
draw = ImageDraw.Draw(img)

r = 5
for vert in unsorted_polygon:
    x = vert[0] * scale_factor
    y = vert[1] * scale_factor
    center_ellipse(x,y,r,'white')
    r *= 1.5
# img.save('pixels.png')


In [803]:
def Calculate_Centroid(polygon):
    x = 0
    y = 0
    for vert in polygon:
        x += vert[0]
        y += vert[1]
    x /= len(polygon)
    y /= len(polygon)

    return [x,y]

def Sort_Vertices(polygon):
    angles = []
    centroid = Calculate_Centroid(polygon)
    vA = np.array([0,1,0])
    for vert in polygon:
        x = vert[0]
        y = vert[1]
        vB = np.array([centroid[0] - x, centroid[1] - y,0])
        
        numerator = np.linalg.norm(np.cross(vA,vB))
        denominator = np.linalg.norm(vA) * np.linalg.norm(vB)
        theta = np.arcsin(numerator / denominator)
        angles.append(theta)

    sorted_verts = [x for _, x in sorted(zip(angles, polygon))]


    return sorted_verts

# print(polygon)
unsorted_sorted = (Sort_Vertices(unsorted_polygon))
sorted_sorted = (Sort_Vertices(polygon))

for x, y in zip(unsorted_sorted,sorted_sorted):
    print(x,y)
# img = Image.open('pixels.png')
# draw = ImageDraw.Draw(img)
# centroid = Calculate_Centroid(unsorted_polygon)
# center_ellipse(centroid[0] * scale_factor,centroid[1] * scale_factor,10,'white')
# img.save('pixels.png')



[50 90] [50 90]
[70 80] [70 80]
[75 20] [75 20]
[15 10] [15 10]
[80 40] [80 40]
[10 50] [10 50]


Drawing seeds

In [800]:
# point_count = 25
# seeds = np.random.rand(point_count,2) * res
# startseeds = seeds

# img = Image.open('pixels.png')
# draw = ImageDraw.Draw(img)

# for seed in seeds:
#     x = seed[0] * scale_factor
#     y = seed[1] * scale_factor
#     center_ellipse(x,y,10,'white')
# img.save('pixels.png')

Creating voronoi diagram

In [801]:
# vor = Voronoi(seeds)